## Hierarchical Content Generation

Hierarchical Content Generation allows you to: 
- create a large amount of content with a small amount input or instructions.
- control the narrative with human directions.
- be consistent with the content
- include previously generated content to make it more holistic 

### Creating a World

In [1]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

In [2]:
world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


In [3]:
from helper import get_together_api_key
print(get_together_api_key())  # Should print the API key


2eafd86cbdfcfe48c936cf08e1804a4310af906b569d78408bd6d779aafb497a


In [4]:
from dotenv import load_dotenv
import os

def get_together_api_key():
    load_dotenv()  # Load the .env file
    api_key = os.getenv("TOGETHER_API_KEY")
    print(f"API Key: {api_key}")  # Debugging line
    return api_key

In [5]:
import os
os.environ["TOGETHER_API_KEY"] = "2eafd86cbdfcfe48c936cf08e1804a4310af906b569d78408bd6d779aafb497a"

from helper import get_together_api_key
print(get_together_api_key())  # Should print the API key

2eafd86cbdfcfe48c936cf08e1804a4310af906b569d78408bd6d779aafb497a


In [6]:
from dotenv import load_dotenv, find_dotenv
import os

# Explicitly find the .env file
dotenv_path = find_dotenv()
print(f"Loaded .env from: {dotenv_path}")
load_dotenv(dotenv_path)

api_key = os.getenv("API_KEYNAME")
print(f"API Key: {api_key}")  # This should print your API key if everything is correct

Loaded .env from: /Users/sarvzz/Desktop/AI-Game-Creator/.env
API Key: None


In [23]:
# Call to together API to generate the world

from together import Together
from helper import get_together_api_key,load_env 

client = Together(api_key=get_together_api_key())

In [24]:
output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",  # Specifying model explicitly
    messages=[
        {"role": "system", "content": system_prompt},  # Defining system message
        {"role": "user", "content": world_prompt}     # User's message
    ]
)

In [26]:
# Print out the created world
world_output =output.choices[0].message.content
print(world_output)

World Name: Kyropeia

World Description: Kyropeia is a realm where ancient, gargantuan creatures known as the "Colossi" roam the land, their massive bodies serving as the foundation for sprawling metropolises. These beasts, born from the earth and infused with primal magic, have been domesticated by humans over centuries, allowing cities to be built upon their backs, shoulders, and even within their hollowed-out bodies. As the Colossi migrate across the landscape, the cities they carry adapt, evolving to harness the creatures' innate powers and forming a symbiotic relationship between beast and builder.


In [31]:
# remove white spaces
world_output = world_output.strip()

# create a world dictionary with 2 keys: 'name' & 'description'
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

### Generating Kingdoms

In [32]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""

In [33]:
output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [34]:
# parse the output for different kingdoms

kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Eldrida" in Kyropeia
Created kingdom "Valtoria" in Kyropeia
Created kingdom "Luminaria" in Kyropeia

Kingdom 1 Description: Luminaria is a mystical kingdom perched upon the shoulders of the luminous Colossus, Elyria. The Luminarians have developed a deep connection with Elyria's radiant energy, allowing them to wield powerful magic and craft wondrous illusions. Led by the benevolent King Orion, Luminaria is a place of wonder and enchantment, where artists and mystics come to tap into the Colossus's essence and unlock the secrets of the universe.


### Generating Towns

In [36]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""

In [37]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.completions.create(
      model="meta-llama/Llama-3-70b-chat-hf",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\n\n'):
        town_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1]\
        .split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name']
        }
        towns[town_name] = town
    kingdom["towns"] = towns

In [39]:
# creating towns for kingdom

for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: \
{town["description"]}')


Creating towns for kingdom: Eldrida...
- Nova Haven created
- Luminaria created
- Ironhollow created

Creating towns for kingdom: Valtoria...
- Khaosgate created
- Ironhaven created
- Emberwatch created

Creating towns for kingdom: Luminaria...
- Lumin's Rest created
- Starhaven created
- Emberhaven created

Town 1 Description: Located on the gentle slope of Elyria's shoulder, Lumin's Rest is a charming town surrounded by lush gardens and sparkling fountains. The town is home to the prestigious Academy of Luminous Arts, where students come to master the intricacies of illusion magic. The town's central square features a magnificent statue of King Orion, and its bustling marketplaces are filled with exotic goods and rare artifacts.


### Generating Non-Player Characters (NPC's)

In [40]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [43]:
# generating npcs

def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
        ],
        # setting to 1 instead of zero, so have more variation in what is created
        temperature=1  
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [44]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
    break


Creating characters for the town of: Nova Haven...
- "Kaelin Darkhunter" created
- "Sofia Argent" created
- "Cassius Eldrid" created

Creating characters for the town of: Luminaria...
- "Kaida Blackwood" created
- "Captain Ryker Stonefist" created
- "Elwyn Moonwhisper" created

Creating characters for the town of: Ironhollow...
- "Kaelin Stonefist" created
- "Lyrien Eldrid" created
- "Arin the Wild" created


In [45]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Ironhollow, Eldrida:
Kaelin is a rugged, battle-hardened inventor from Ironhollow, with a thick beard and arms covered in soot and oil stains. He wears a leather apron over his worn clothing, and his eyes gleam with a passion for creation. As a master craftsman, Kaelin has dedicated his life to harnessing Arkeia's power to build machines that can protect the people of Ironhollow from the dangers of the surface world. However, his drive for innovation is motivated by a deep-seated fear of losing loved ones to the Colossi's unpredictable migrations.


### Save our World

In [48]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

save_world(world, '/Users/sarvzz//Desktop/AI-Game-Creator/Kyropeia.json')